# 強化学習におけるリプシッツ連続性

参考：
* [Lipschitz Continuity in Model-based Reinforcement Learning](https://arxiv.org/abs/1804.07193)

今回は状態空間が連続である場合の話について学びます．
状態空間が離散の場合は価値反復法が使えますが，連続の場合には使えません．

しかし，MDPの中にリプシッツ連続性のようなものが導入されてる場合は解くことができます．見てみましょう．

## リプシッツ連続性とWasserstein距離

数学的な準備をします．

---

**リプシッツ連続性**

２つの距離空間$\left(M_1, d_1\right)$と$\left(M_2, d_2\right)$を考えます．
関数$f: M_1 \mapsto M_2$は次で定義されるリプシッツ定数が有限である時にリプシッツ連続である，といいます．

$$
K_{d_1, d_2}(f):=\sup _{s_1 \in M_1, s_2 \in M_1} \frac{d_2\left(f\left(s_1\right), f\left(s_2\right)\right)}{d_1\left(s_1, s_2\right)}
$$

これは次と等価です：

$$
\forall s_1, \forall s_2 \quad d_2\left(f\left(s_1\right), f\left(s_2\right)\right) \leq K_{d_1, d_2}(f) d_1\left(s_1, s_2\right)
$$

$f$は$K_{d_1, d_2}(f)=1$のときにnon-expansionと呼び，$K_{d_1, d_2}(f)<1$のときにcontractionと呼ばれます．

---

**一様リプシッツ連続**

関数$f: M_1 \times \mathcal{A} \mapsto M_2$は次の定数が有限であるときに$\mathcal{A}$について一様リプシッツ連続であるといいます．

$$
K_{d_1, d_2}^{\mathcal{A}}(f):=\sup _{a \in \mathcal{A}} \sup _{s_1, s_2} \frac{d_2\left(f\left(s_1, a\right), f\left(s_2, a\right)\right)}{d_1\left(s_1, s_2\right)}
$$

---

**Wasserstein距離**

距離空間$(M, d)$と，$M$上の全ての確率測度$\mathbb{P}(M)$について，確率分布$\mu_1 \in \mathbb{P}(M)$と$\mu_2 \in \mathbb{P}$のWasserstein距離は，

$$
W\left(\mu_1, \mu_2\right):=\inf _{j \in \Lambda} \iint j\left(s_1, s_2\right) d\left(s_1, s_2\right) d s_2 d s_1
$$

で定義さます．
ここで，$j$は$M\times M$上の分布であり，$\mu_1(s_1)=\sum_{s_2} j(s_1, s_2)$および$\mu_2(s_2)=\sum_{s_1} j(s_1, s_2)$を満たします．
$\Lambda$はそのような$j$の全ての集合です．

**解釈**

距離$d(s_1, s_2)$は$s_1$を$s_2$に輸送するコストとみなすことができます．
よって，積分$\iint j\left(s_1, s_2\right) d\left(s_1, s_2\right) d s_2 d s_1$は測度$\mu_1$を$\mu_2$に動かすコストとみなせます．

このため，Wasserstein距離はEarth-Mover's Distanceとも呼ばれます．

**双対性**

Wasserstein距離は双対性を使うと，次のようなリプシッツ連続性を使った形式に変形できます．

$$
W\left(\mu_1, \mu_2\right)=\sup _{f: K_{d, d_{\mathbb{R}}}(f) \leq 1} \int\left(f(s) \mu_1(s)-f(s) \mu_2(s)\right) d s .
$$

これはKantorovich-Rubinstein双対性と呼ばれます．

---


## リプシッツ連続性を使ったMDPの定義

次の定義を導入します：

---

距離が導入されている状態空間$\left(\mathcal{S}, d_{\mathcal{S}}\right)$と，行動空間$\mathcal{A}$を考えます．
$F_g=\{f: \mathcal{S} \mapsto \mathcal{S}\}$を状態から状態へmapする関数の集合とします．
また，$a \in \mathcal{A}$について，この上の分布$g(f\mid a)$を考えます．

このとき，次の定数が有限のとき，$F_g$はLipschitz model classであるといいます．

$$
K_F:=\sup _{f \in F_g} K_{d_{\mathcal{S}}, d_{\mathcal{S}}}(f)
$$

つまり，$F_g$の中の関数全てがリプシッツ連続である場合です．

---

**一般的な遷移関数**

ここで定義した$F_g$を使えば，遷移関数を定義することができます：

$$
\widehat{T}\left(s^{\prime} \mid s, a\right)=\sum_{f\in F_g} \mathbb{1}\left(f(s)=s^{\prime}\right) g(f \mid a) .
$$

さらに，遷移元を状態ではなく，分布$\mu(s)$を考えた場合の一般的な遷移関数も定義しておきます：

$$
\widehat{T}_{\mathcal{G}}\left(s^{\prime} \mid \mu, a\right)=\int_s \underbrace{\sum_f \mathbb{1}\left(f(s)=s^{\prime}\right) g(f \mid a)}_{\widehat{T}\left(s^{\prime} \mid s, a\right)} \mu(s) d s .
$$

実はこのような形式を使うと，有限MDPの遷移関数は全て表現できます．（つまり，この定義自体では$\mathcal{S}$が距離空間である必要はありません．距離の導入は後で使います．）

**証明**

$\operatorname{Pr}\left(s, a, s^{\prime}\right)$を有限MDPの遷移関数とします．
状態に次のような順番を導入しましょう：$s_1, \ldots, s_n$．また，$s_0$を到達不可能な状態とします．
これを使って，次の累積分布を導入します．

$$
C\left(s, a, s_i\right):=\sum_{j=0}^i \operatorname{Pr}\left(s, a, s_j\right)
$$

次の$L$を定義します（$C$がとり得る値の集合です）：

$$
L:=\left\{C\left(s, a, s_i\right) \mid \quad s \in \mathcal{S}, i \in[0, n]\right\}
$$

MDPは有限なので，$|L|$は有限です．
$c_i$を$L$のうち，$i$番目に小さい値とします．このとき$c_0=0$かつ$c_{|L|}=1$です．

これを使って，$f_1, \ldots, f_{|L|}$を次のように定義します．

任意の$\forall i \in [|L|]$と$\forall j \in n$について，
$$
C\left(s, a, s_{j-1}\right)<c_i \leq C\left(s, a, s_j\right)
$$
のとき，またそのときに限り，$f_i(s)=s_j$とします．

また，$f$上の分布を

$$
g\left(f_i \mid a\right):=c_i-c_{i-1} .
$$

とします．こうすると，

$$
\begin{aligned}
& \sum_i \mathbb{1}\left(f_i(s)=s_j\right) g\left(f_i \mid a\right) \\
& =\sum_i \mathbb{1}\left(C\left(s, a, s_{j-1}\right)<c_i \leq C\left(s, a, s_j\right)\right)\left(c_i-c_{i-1}\right) \\
& =C\left(s, a, s_j\right)-C\left(s, a, s_{j-1}\right) \\
& =\operatorname{Pr}\left(s, a, s_j\right)
\end{aligned}
$$

なので，結局遷移関数が復元できます．


---


## Wasserstein距離を使った遷移関数の評価

強化学習ではしばしば，遷移関数$T$と推定した遷移関数$\hat{T}$の差が重要になることがあります（モデルベースRLなど）．
よく使われるのは全変動距離やKLダイバージェンスですが，状態空間に距離が定義されている場合はWasserstein距離を使ったほうが自然に差が測定できます．

（これは面倒なので論文の４章参照）

続いて，Wasserstein距離が離れた遷移関数同士で誤差がどのようになるか見てみましょう．まず，次の定義を導入します：

$$
\forall s \forall a \quad W(\widehat{T}(\cdot \mid s, a), T(\cdot \mid s, a)) \leq \Delta
$$

を満たすような$\hat{T}$が$F_g$によって誘導されるとき，$F_g$は$\Delta$-accurateであるといいます．
さて，行動の系列$a_0, \ldots, a_{n-1}$について，$\widehat{T}_{\mathcal{G}}^n(\cdot \mid \mu):=\underbrace{\widehat{T}_{\mathcal{G}}\left(\cdot \mid \widehat{T}_{\mathcal{G}}\left(\cdot \mid \ldots \widehat{T}_{\mathcal{G}}\left(\cdot \mid \mu, a_0\right) \ldots, a_{n-2}\right), a_{n-1}\right)}_{n \text { recursive calls }}$として，遷移の誤差

$$
\delta(n):=W\left(\widehat{T}_{\mathcal{G}}^n(\cdot \mid \mu), T_{\mathcal{G}}^n(\cdot \mid \mu)\right)
$$

について考えましょう．


このとき，$\Delta$-accurate かつリプシッツ定数が$K_F$の$\widehat{T}_{\mathcal{G}}$と，リプシッツ定数が$K_T$の真の遷移関数$T_\mathcal{G}$について，

$$
\delta(n):=W\left(\widehat{T}_{\mathcal{G}}^n(\cdot \mid \mu), T_{\mathcal{G}}^n(\cdot \mid \mu)\right) \leq \Delta \sum_{i=0}^{n-1}(\bar{K})^i
$$

が成立します．ここで，$\bar{K}=\min\{K_F, K_T\}$です．

つまり，真の遷移との誤差を小さくするためには，遷移がリプシッツ連続であることと，１ステップの誤差が小さいことが重要になります．

<!-- また，次が成立します． -->
<!-- 
$\hat{T}_\mathcal{G}$がLipschitz model class $F_g$によって誘導されるとき，$\hat{T}_\mathcal{G}$は次の定数について一様リプシッツ連続です．

$$
K_{W, W}^{\mathcal{A}}\left(\widehat{T}_{\mathcal{G}}\right):=\sup _a \sup _{\mu_1, \mu_2} \frac{W\left(\widehat{T}_{\mathcal{G}}\left(\cdot \mid \mu_1, a\right), \widehat{T}_{\mathcal{G}}\left(\cdot \mid \mu_2, a\right)\right)}{W\left(\mu_1, \mu_2\right)} \leq K_F
$$

 -->


## Lipschitz modelにおける価値の誤差解析

行動空間を$\mathcal{A}=\{a\}$とし，
$\langle\mathcal{S}, \mathcal{A}, T, R, \gamma\rangle$と$\langle\mathcal{S}, \mathcal{A}, T, R, \gamma\rangle$なるMDP２つを考えます．
これはつまり固定された方策によって遷移が確定したマルコフ連鎖と同じですね．

そして，

$$
\begin{aligned}
&V_T(s):=\sum_{n=0}^{\infty} \gamma^n \int T_{\mathcal{G}}^n\left(s^{\prime} \mid \delta_s\right) R\left(s^{\prime}\right) d s^{\prime}\\
&V_{\widehat{T}}(s):=\sum_{n=0}^{\infty} \gamma^n \int \widehat{T}_{\mathcal{G}}^n\left(s^{\prime} \mid \delta_s\right) R\left(s^{\prime}\right) d s^{\prime}
\end{aligned}
$$

なる状態価値を考えましょう．これもいつもの状態価値そのまんまですね．
このとき，次が成立します．

$F_g$がLipschitz model classであるとし，$\hat{T}$は$\Delta$-accurateとします．また，報酬関数のリプシッツ定数が$K_R=K_{d_{\mathcal{S}}, \mathbb{R}}(R)$であるとします．このとき，$\forall s \in \mathcal{S}$ and $\bar{K} \in\left[0, \frac{1}{\gamma}\right)$について，

$$
\left|V_T(s)-V_{\widehat{T}}(s)\right| \leq \frac{\gamma K_R \Delta}{(1-\gamma)(1-\gamma \bar{K})}
$$

が成立します．
つまり，異なる遷移によって計算された価値の差をバウンドするには，MDPのリプシッツ性が重要になってきます．